## Flower Classifier with VGG16 on Oxford17 Flowers Dataset

In [1]:
from keras.applications import VGG16

# VGG16 works on 224 x 224 pixel image input data
img_rows = 224
img_cols = 224

vgg16 = VGG16(weights = 'imagenet',
             include_top = False,
             input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.
/home/naren/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/naren/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/naren/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/naren/.local/lib/python3.6/site-packages/tensorflow/python/fra


58892288/58889256 [==============================] - 34s 1us/step


In [2]:
for layer in vgg16.layers:
    layer.trainable = False

In [7]:
def addTopModel(bottom_model, num_classes, D = 256):
    top_model = bottom_model.output
    top_model = Flatten(name = 'flatten')(top_model)
    top_model = Dense(D, activation = 'relu')(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs = vgg16.input, outputs = FC_Head)

print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = './17_flowers/train'
validation_data_dir = './17_flowers/validation'

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_batchsize = 16
val_batchsize = 16

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = train_batchsize,
    class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = val_batchsize,
    class_mode = 'categorical',
    shuffle = False)

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [14]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('./flowers_vgg16.h5',
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta = 0,
                         patience = 3,
                         verbose = 1,
                         restore_best_weights = True)

callbacks = [checkpoint, earlystop]

model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr = 0.001),
             metrics = ['accuracy'])

nb_train_samples = 1190
nb_validation_samples = 170

epochs = 20
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs =epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save('./flowers_vgg16.h5')


Epoch 1/20
74/74 [==============================] - 811s 11s/step - loss: 3.7717 - accuracy: 0.1831 - val_loss: 3.6212 - val_accuracy: 0.3875

Epoch 00001: val_loss improved from inf to 3.62125, saving model to ./flowers_vgg16.h5
Epoch 2/20
74/74 [==============================] - 850s 11s/step - loss: 2.1049 - accuracy: 0.3578 - val_loss: 0.4074 - val_accuracy: 0.7013

Epoch 00002: val_loss improved from 3.62125 to 0.40743, saving model to ./flowers_vgg16.h5
Epoch 3/20
74/74 [==============================] - 853s 12s/step - loss: 1.6960 - accuracy: 0.4617 - val_loss: 0.6575 - val_accuracy: 0.6818

Epoch 00003: val_loss did not improve from 0.40743
Epoch 4/20
74/74 [==============================] - 822s 11s/step - loss: 1.4217 - accuracy: 0.5358 - val_loss: 1.2579 - val_accuracy: 0.7727

Epoch 00004: val_loss did not improve from 0.40743
Epoch 5/20
74/74 [==============================] - 696s 9s/step - loss: 1.2671 - accuracy: 0.5963 - val_loss: 0.1519 - val_accuracy: 0.7857

Epoch

In [15]:
from keras.models import load_model

classifier = load_model("./flowers_vgg16.h5")